# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [53]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [54]:
# !pip install --upgrade numpy
# !pip install --upgrade pandas


In [55]:
import pandas as pd

In [56]:
df = pd.read_csv('./data/Data.csv')

In [57]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [58]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [59]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [60]:
prompt = ChatPromptTemplate.from_template( "What makes this {Product} so perfect?"
)

In [61]:
chain = LLMChain(llm=llm, prompt=prompt)

In [62]:
product = "Pillows Insert"
print(chain.run(product))

- High quality materials: The insert is made with high quality, durable materials that ensure long-lasting comfort and support.
- Perfect size: The insert is designed to fit perfectly inside standard pillow cases, providing a seamless and snug fit.
- Plush and fluffy: The insert is filled with plush, fluffy material that provides a soft and comfortable feel for a good night's sleep.
- Hypoallergenic: The insert is hypoallergenic, making it suitable for those with allergies or sensitive skin.
- Easy to care for: The insert is easy to care for and can be machine washed and dried for convenience.
- Versatile: The insert can be used in various settings, such as on a bed, couch, or chair, making it a versatile and practical addition to any home.


## SimpleSequentialChain

In [63]:
from langchain.chains import SimpleSequentialChain

In [64]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template( "What makes this {Product} so perfect?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [65]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template( 
    "Using the features: {input}, write a marketing tagline for this product."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [66]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [67]:
print(overall_simple_chain.run(product))



> Entering new SimpleSequentialChain chain...
There are several reasons why this Pillows Insert is considered perfect:

1. Materials: The insert is made from high-quality and durable materials, such as hypoallergenic polyester fiber filling or down alternative, ensuring comfort and longevity.

2. Size options: The insert comes in a variety of sizes to fit different pillow covers perfectly, providing a seamless and polished look.

3. Supportive: The insert provides excellent support and shape retention, ensuring that the pillow stays fluffy and comfortable over time.

4. Easy maintenance: The insert is easy to care for, as it is machine washable and dries quickly, making it convenient to keep clean.

5. Versatile: The insert is versatile and can be used in a variety of pillow covers and decorative throw pillows, making it a great addition to any home décor.
"Elevate your comfort and style with our perfect Pillows Insert - the ultimate blend of quality, support, and versatility!"

> Fi

**Repeat the above twice for different products**

In [72]:
# Run the chain for a product
product = "Luxury Air Mattress"
result = overall_simple_chain.run(product)
print("Marketing tagline for product:", result)

# Repeat for another product
product = "Milk Frother Handheld"
result = overall_simple_chain.run(product)
print("Marketing tagline for product:", result)



> Entering new SimpleSequentialChain chain...
1. Superior comfort: This luxury air mattress is designed with high-quality materials and advanced technology to provide maximum comfort and support for a restful night's sleep.

2. Durability: Made with durable materials, this air mattress is built to last and withstand frequent use without losing its shape or firmness.

3. Adjustable firmness: The air mattress comes with a built-in pump that allows you to easily adjust the firmness level to your preference, ensuring a personalized sleep experience.

4. Easy to use: The air mattress inflates and deflates quickly and easily, making it convenient to set up and pack away for storage or travel.

5. Portable: This luxury air mattress is lightweight and compact when deflated, making it easy to transport and set up in any room or location.

6. Versatile: Whether you're hosting guests, going camping, or need a temporary sleeping solution, this luxury air mattress provides a comfortable and conve

## SequentialChain

In [68]:
from langchain.chains import SequentialChain

In [73]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template( "Translate this review to French: {Review}"
          
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "review_translation"
                    )


In [75]:
second_prompt = ChatPromptTemplate.from_template( "Summarize this review: {review_translation}",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "review_tanslation_summarized"
                    )


In [76]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the {review_tanslation_summarized} back to English."
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key= "review_tanslation_summarized_eng"
                      )


In [77]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow-up message to the customer based on this summary in German: {review_tanslation_summarized_eng}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key= "follow_up_message"
                     )


In [78]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["Review"],
    output_variables=["review_translation", "review_tanslation_summarized", "review_tanslation_summarized_eng", "follow_up_message"],
    verbose=True
)

In [ ]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...


**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**